# Quora Insincere Questions Sklearn Baseline Models with Downsampling
In this kernel we will try several of sklearn's classification algorithms (plus XGBoost) to find a good baseline.  We will then select the best performing models and try to improve performance by adjusting hyperparameters and features.

In [1]:
# import packages
import numpy as np
import pandas as pd

import spacy
import re

from gensim import corpora, models, similarities

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

np.random.seed(27)

In [3]:
train = pd.read_csv('../SUDHEER/train.csv')
test = pd.read_csv('../SUDHEER/test.csv')
train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


#### Text Pre-processing and Downsampling
Because we have an imbalanced dataset we will downsample the majority class to equal the size of the minority class.  This will not only balance our dataset, but will decrease processing time due to the decreased number of samples in the training data.

In [4]:
# taking a small sample (with downsampling of majority class) of the training data to speed up processing
from sklearn.utils import resample

sincere = train[train.target == 0]
insincere = train[train.target == 1]

train = pd.concat([resample(sincere,
                     replace = False,
                     n_samples = len(insincere)), insincere])

In [5]:
contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(contractions.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return contractions[match.group(0)]
    return c_re.sub(replace, text)

In [6]:
# function to clean and lemmatize text and remove stopwords
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_numeric
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, strip_short

CUSTOM_FILTERS = [lambda x: x.lower(), #lowercase
                  strip_tags, # remove html tags
                  strip_punctuation, # replace punctuation with space
                  strip_multiple_whitespaces,# remove repeating whitespaces
                  strip_non_alphanum, # remove non-alphanumeric characters
                  strip_numeric, # remove numbers
                  remove_stopwords,# remove stopwords
                  strip_short # remove words less than minsize=3 characters long
                 ]
nlp = spacy.load('en')

def gensim_preprocess(docs, logging=True):
    docs = [expandContractions(doc) for doc in docs]
    docs = [preprocess_string(text, CUSTOM_FILTERS) for text in docs]
    texts_out = []
    for doc in docs:
    # https://spacy.io/usage/processing-pipelines
        doc = nlp((" ".join(doc)),  # doc = text to tokenize => creates doc
                  # disable parts of the language processing pipeline we don't need here to speed up processing
                  disable=['ner', # named entity recognition
                           'tagger', # part-of-speech tagger
                           'textcat', # document label categorizer
                          ])
        texts_out.append([tok.lemma_ for tok in doc if tok.lemma_ != '-PRON-'])
    return pd.Series(texts_out)

gensim_preprocess(train.question_text.iloc[10:15])

0                  [invent, modern, portfolio, theory]
1    [place, sleep, amsterdam, airport, night, arrive]
2                      [expansion, general, kth, term]
3                        [accomodation, month, berlin]
4    [buddhism, popular, religion, india, religion,...
dtype: object

In [11]:
# apply text-preprocessing function to training set
%time train_corpus = gensim_preprocess(train.question_text)

Wall time: 15min 8s


In [12]:
# create ngrams
ngram_phraser = models.Phrases(train_corpus, threshold=1)
ngram = models.phrases.Phraser(ngram_phraser)
#print example
print(ngram[train_corpus[0]])

# apply model to corpus
texts = [ngram[token] for token in train_corpus]

['well', 'learn', 'develope', 'app', 'linux']


In [13]:
# preparing ngrams for modeling
texts = [' '.join(text) for text in texts]
train['ngrams'] = texts
train.head()

,qid,question_text,target,ngrams
243613,2fa4ba03c124bf94fdd8,What is the best to learn developing apps on L...,0,well learn develope app linux
383126,4b16b43ccee0d0375d73,What is the indirect competitors of Starbucks?,0,indirect competitor starbucks
995356,c30ee49476e64839b1c9,How are high level nuclear waste being dispose...,0,high_level nuclear_waste dispose worldwide
737244,9063754fa48fa7c9eed4,What is so great about Google's Associate Prod...,0,great google associate product_manager http_ww...
44857,08c89ffca45ed16018f6,"Which IIT to choose for mtech CSE, IIT Kanpur,...",0,iit choose mtech cse_iit kanpur iit kgp iit_ma...


In [14]:
# represent features as BOW
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(train.ngrams)

# split into test and train sets
X_train, X_test, y_train, y_test = train_test_split(train.ngrams, train.target, test_size=0.2)

## Logistic Regression Baseline Model
Logistic Regression is a linear model for classification.  They are fast to train and predict, scale well, and are easy to interpret, and are therefore a good choice for a baseline model.

In [15]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(vectorizer.transform(X_train), y_train)

print('Logistic Regression Score: ', lr.score(vectorizer.transform(X_test), y_test))

C:\Users\SUDHEER\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Logistic Regression Score:  0.867559707956936


In [16]:
y_ = lr.predict(vectorizer.transform(X_test))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87     16187
           1       0.88      0.85      0.86     16137

    accuracy                           0.87     32324
   macro avg       0.87      0.87      0.87     32324
weighted avg       0.87      0.87      0.87     32324



In [17]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, y_))

,0,1
0,14375,1812
1,2469,13668


## Interpreting the Results - Classification Report

* **Precision** is the number of true positives divided by all positive predictions. Precision is also called Positive Predictive Value. It is a measure of a classifier's exactness. Low precision indicates a high number of false positives.
* **Recall** is the number of true positives divided by the number of positive values in the test data. Recall is also called Sensitivity or the True Positive Rate. It is a measure of a classifier's completeness. Low recall indicates a high number of false negatives.
* **F1-Score** is the harmonic mean of precision and recall.
* **Support** is the number of true results for each class.

## Bernoulli Naive Bayes Baseline Model
Naive Bayes classifiers are super fast to train and work well with high-dimensional sparse data, including text. They are based on applying [Bayes' Theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem) and are 'naive' in that they assume independence between features.  Scikit-Learn implements several types of Naive Bayes classifiers that are widely used for text data including Bernoulli (which we use here) and Multinomial.

In [18]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
%time bnb.fit(vectorizer.transform(X_train), y_train)

print('Naive Bayes Score: ', bnb.score(vectorizer.transform(X_test), y_test))

Wall time: 2.45 s
Naive Bayes Score:  0.8657653755723301


In [19]:
%time bnb_y_ = bnb.predict(vectorizer.transform(X_test))

print(classification_report(y_test, bnb_y_))

Wall time: 559 ms
              precision    recall  f1-score   support

           0       0.88      0.85      0.86     16187
           1       0.85      0.88      0.87     16137

    accuracy                           0.87     32324
   macro avg       0.87      0.87      0.87     32324
weighted avg       0.87      0.87      0.87     32324



In [20]:
pd.DataFrame(confusion_matrix(y_test, bnb_y_))

,0,1
0,13711,2476
1,1863,14274


## XGBoost Model
XGBoost (Extreme Gradient Boosting) is an implementation of gradient boosted decision trees designed for speed and performance.  As such, it often outperforms other algorithms.

In [21]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier().fit(vectorizer.transform(X_train), y_train)

print('XGBoost Score: ', xgb_model.score(vectorizer.transform(X_test), y_test))

XGBoost Score:  0.7004393020665759


In [22]:
xgb_y_ = xgb_model.predict(vectorizer.transform(X_test))

print(classification_report(y_test, xgb_y_))

              precision    recall  f1-score   support

           0       0.64      0.93      0.76     16187
           1       0.86      0.47      0.61     16137

    accuracy                           0.70     32324
   macro avg       0.75      0.70      0.68     32324
weighted avg       0.75      0.70      0.68     32324



In [23]:
pd.DataFrame(confusion_matrix(y_test, xgb_y_))

,0,1
0,14984,1203
1,8480,7657


## Ensemble Model
[Scikit-learn](https://scikit-learn.org/stable/modules/ensemble.html#ensemble) states that "the goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator."  We will combine our above three models using scikit-learn's voting classifier.

In [24]:
from sklearn.ensemble import VotingClassifier

#create submodels
estimators = []

model1 = lr
model2 = bnb
model3 = xgb_model


estimators.append(('logistic', model1))
estimators.append(('bernoulli', model2))
estimators.append(('xgboost', model3))


# create ensemble model
%time ensemble = VotingClassifier(estimators).fit(vectorizer.transform(X_train), y_train)
print('Ensemble Score: ', ensemble.score(vectorizer.transform(X_test), y_test))

C:\Users\SUDHEER\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Wall time: 1min 3s
Ensemble Score:  0.8652085137977973


In [25]:
ensemble_y_ = ensemble.predict(vectorizer.transform(X_test))

print(classification_report(y_test, ensemble_y_))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87     16187
           1       0.89      0.84      0.86     16137

    accuracy                           0.87     32324
   macro avg       0.87      0.87      0.87     32324
weighted avg       0.87      0.87      0.87     32324



In [26]:
pd.DataFrame(confusion_matrix(y_test, ensemble_y_))

,0,1
0,14434,1753
1,2604,13533


## Interpret Results
Logistic Regression F1: 86.9  
Naive Bayes F1: 86.5  
XGBoost F1: 70.9  
Ensemble F1: 86.6  



In [35]:
# preprocessing/lemmatizing/stemming test data
%time test_corpus = gensim_preprocess(test.question_text)
test_texts = [ngram[token] for token in test_corpus]

test_texts = [' '.join(text) for text in test_texts]
test['ngrams'] = test_texts
test.head()

Wall time: 15min 50s


,qid,question_text,ngrams
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...,woman rude_arrogant little_bite wealth power
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...,apply_college engineer bms college_engineer wa...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...,like nurse practitioner
3,000086e4b7e1c7146103,Who are entrepreneurs?,entrepreneur
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?,education make good people_nowadays


In [36]:
#ensemble on test data
ensemble.fit(vectorizer.transform(train.ngrams), train.target)
prediction = ensemble.predict(vectorizer.transform(test.ngrams))

submission = pd.DataFrame({'qid':test.qid, 'prediction':prediction})
submission.to_csv('submission.csv', index=False)
submission.head()

C:\Users\SUDHEER\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,qid,prediction
0,0000163e3ea7c7a74cd7,1
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0
